# Contents

## 1. 데이터셋 확인

- 1.1 Null data check    
- 1.2 Target label 확인    

## 2. Exploratory data analysis
- 2.1 Pclass    
- 2.2 Sex    
- 2.3 Both Sex and Pclass   
- 2.4 Age      
- 2.5 Pclass, Sex, Age   
- 2.6 Embarked    
- 2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)    
- 2.8 Cabin  


- 타이타닉은 알다시피, 사상 최대 해난사고로써, 1,500여명의 희생자가 생겼다. 
- 그래서 이 튜토리얼을 이용하여 타이타닉에 탑승한 사람들의 신상정보를 이용및 활용할 것이다.
    - 그정보를 토대로 승선한 사람들의 생존 여부를 예측하는 모델을 생성할 것이다. 

- 여기서는 여러가지 시각화 도구, 데이터 분석도구, 머신러닝도구를 사용할 것이다. 
    - 시각화도구 (matplotlib, seaborn, plotly)
    - 데이터 분석 도구(pandas, numpy)
    - 머신 러닝 도구(sklearn)

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

# 스타일 
plt.style.use('seaborn')
sns.set(font_scale=2.5)
# matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅
# 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편하다.

import missingno as msno

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

- 앞으로 진행할 프로세스 
    - 데이터 셋 확인 
        - 대부분의 캐글 데이터들이 잘 정제 되어 있다. 
        - 하지만 가끔 null data가 존재 
             - 이것을 확인하고 수정 
    - 탐색적 데이터 분석(exploratory data analysis)
        - 여러 feature들을 개별적으로 분석하고, feature들 간의 상관관계를 확인 
        - 여러가지 시각화 툴을 사용하여 인사이트를 얻는다.
    - feature engineering 
        - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature들을 engineering한다. 
        - one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리등등....
    - model 만들기 
        - sklearn을 사용해 모델을 만든다.
        - 파이썬에서 머신러닝을 할 때에는 sklearn 을 사용
            - 수많은 알고리즘을 일관된 문법으로 사용가능하다. 
    - 모델 학습 및 예측 
        - train_set을 가지고 모델을 학습시킨 후, test_set을 가지고 예측(prediction)을 진행
    - 모델 평가 
        - 예측성능이 원하는 수준인지 판단
        - 풀려는 문제에 따라 모델을 평가하는 방식도 달라진다. 
            - 학습된 모델이 어떤것을 학습하였는지 확인이 필요 

# 1. Data_set 확인 

- 파이썬에서 테이블화 된 데이터를 다루는데 가장 최적화되어 있으며, 많이 쓰는 라이브러리는 pandas
- pandas를 사용하여 데이터 셋의 간단한 통계적 분석을 시작으로 복잡한 처리들을 간단한 메소드를 사용하여 해낼 수 있다. 
- 파이썬으로 데이터 분석을 한다고 하면 반드시 능숙해져야 할 라이브러리이기 때문에 여러 커널을 공부하면서 사용법에 능숙해지도록 반복하기를 권장한다. 

In [ ]:
# 캐글에서 데이터 셋은 보통  train, testset로 나누어져 있다. 

df_train = pd.read_csv('../../input/train.csv')
df_test = pd.read_csv('../../input/test.csv')

In [ ]:
df_train.head()

### 우리가 다루는 문제에서 feature는  Pclass, Age, SibSp, Parch, Fare 
### -> 예측하려는 타겟 레이블은 Survived 

|변수(feature, variable)|정의|설명|타입|
|------|---|---|---|
|survival|생존여부 |target label 임. 1, 0 으로 표현됨|	integer|
|Pclass  |티켓의 클래스	1 = 1st, 2 = 2nd, 3 = 3rd 클래스로 나뉘며 |1 = 1st, 2 = 2nd, 3 = 3rd 클래스로 나뉘며 categorical feature|	integer|
|sex     |성별     |male, female 로 구분되며 binary	|string|
|Age     |나이     |continuous|integer|
|sibSp   |함께 탑승한 형제와 배우자의 수|quantitative|integer|
|parch   |함께 탑승한 부모, 아이의 수	 |quantitative|integer|
|ticket  |티켓 번호|alphabat + integer  |string|
|fare    |탑승료	|continuous          |float|
|cabin   |객실 번호|alphabat + integer  |string|
|embared |탑승 항구|C = Cherbourg, Q = Queenstown, S = Southampton|string|

---
- pandas dataframe 에는 describe() 메소드가 있는 데, 이를 쓰면 각 feature 가 가진 통계치들을 반환

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

- 테이블에서 보다시피  PassenserID 숫자와 다른, 결측치가 존재하는 열을 추측할 수 있다.
- 이를 정확하게 확인하기 위해 시각화 해서 살펴 볼게요 

# 1.1 결측치 확인 

In [ ]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100* (df_train[col].isnull().sum()/ df_train[col].shape[0]))
    print(msg)

In [ ]:
# 이해하기 쉽게 정리 
for col in df_train.columns:
    persent = 100* (df_train[col].isnull().sum()/ df_train[col].shape[0])
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, persent)
    print(msg)

In [ ]:
for col in df_test.columns:
    persent = 100* (df_test[col].isnull().sum()/ df_test[col].shape[0])
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, persent)
    print(msg)

- 학습, 테스트 셋에서 age(둘다20%), cabin(둘다 약 80%),Embarked(Train만 0.22%)
    - 결측치가 존재하는것을 알 수 있다. 
- 결측치를 시각화 하기 위해서 MANO라는 라이브러리를 사용하면 null값을 좀 더 쉽게 확인이 가능하다. 

In [ ]:
# msno.matrix(df= df_train, figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
msno.matrix(df= df_train.iloc[:,:],  figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
# 코드 확인용 
msno.matrix(df= df_train.iloc[:,:7],  figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
msno.bar(df=df_train.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
# 코드 확인용 
msno.bar(df=df_train.iloc[:,:5], figsize=(8,8), color=(0.8, 0.5, 0.2))

# 1.2 타겟 데이터 확인 

- 타겟 레이블이 어떤 분포(distribution)를 가지고 있는지 확인 
- 바이너리 분류( binary classification )
    - 1과 0의 분포가 어떻게 분포하느냐에 따라 모델의 평가 방법이 달라진다. 
    

In [ ]:
f,ax = plt.subplots(1,2, figsize=(18,8))

df_train.Survived.value_counts().plot.pie(explode=[0, 0.1], autopct="%1.1f%%", ax= ax[0] , shadow= True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

In [ ]:
df_train.Survived.value_counts()

In [ ]:
f,ax = plt.subplots(1,2, figsize=(18,8))

Survived_cunt = df_train.Survived.value_counts()
Survived_cunt.plot.pie(explode=[0, 0.1], autopct="%1.1f%%", ax= ax[0])

sns.countplot('Survived', data=df_train, ax=ax[1])

plt.show()

- 죽은 사람이 많다는것을 확인 가능 
- 38.4% 가 살아 남았다. 

- 타겟 레이블의 분포가 제법 균일한것도 확인 
- 불균일한 경우 예를들면 100중 1이 99, 0이 1개인 경우에는 만약 모델이 모든것을 1이라고 해도 
- 정확도가 99%가 나온다 
- 0을 찾는 문제라면 이모델은 원하는 결과를 줄 수 없다. -> 지금은 중요하지 않음  

# 2. EDA(Exploratory data analysis)

- 본격적인 데이터 분석을 진행 -> 적절한 시각화가 필요
- 시각화 라이브러리 matplotlib, seaborn, plotly -> 특정 목적에 맞게 정리해두면 좋다.


# 2.1 Pclass

- Pclass는 ordinal, 서수형 데이터 -> 카테고리인 동시에 순서가 있는 데이터 타입 
- 먼저 Pclass에 따른 생존률의 차이를 살펴볼것이다. 
    - 엑셀의 피벗 차트와 유사한 작업을 한다.
    - Pandas dataframe 에서는 groupby 를 사용, 혹은 pivot 이라는 메소드를 이용 

- 'Pclass','Survived'를 가지고온 후 Pclass로 묶는다 
    - 각 Pclass마다 0,1 이 카운드 되는데 이것을 평균을 내면 각 Pclass별 생존률이 나온다.

- 아래와 같이 count()메소드를 이용하면, 각 class에 몇명이 있는지 확인이 가능하다 
    - sum()메소드를 이용하면 216 명중 생존한(survived=1)사람의 총합을 얻는다. 


In [ ]:
check_test = df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True)
check_test

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True).count()

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True).sum()
# 뭐가 다른지 확실히 인지해야해

# 크로스 탭(교차테이블)

크로스 탭
- 범주형 변수로 되어있는 요인(factors)별로 교차분석(cross tabulations)
- 행, 열 요인 기준 별로 빈도를 세어서 도수분포표(frequency table)
    -  행과 열 위치에는 array 형식의 데이터가 들어감 
- 교차표(contingency table) 를 만들어준다. 


```py 
pandas.crosstab
pandas.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, margins_name: str = 'All', dropna: bool = True, normalize=False) → 'DataFrame'[source]
Compute a simple cross tabulation of two (or more) factors. By default computes a frequency table of the factors unless an array of values and an aggregation function are passed.

Parameters
indexarray-like, Series, or list of arrays/Series
Values to group by in the rows.

columnsarray-like, Series, or list of arrays/Series
Values to group by in the columns.

valuesarray-like, optional
Array of values to aggregate according to the factors. Requires aggfunc be specified.

rownamessequence, default None
If passed, must match number of row arrays passed.

colnamessequence, default None
If passed, must match number of column arrays passed.

aggfuncfunction, optional
If specified, requires values be specified as well.

marginsbool, default False
Add row/column margins (subtotals).

margins_namestr, default ‘All’
Name of the row/column that will contain the totals when margins is True.

New in version 0.21.0.

dropnabool, default True
Do not include columns whose entries are all NaN.

normalizebool, {‘all’, ‘index’, ‘columns’}, or {0,1}, default False
Normalize by dividing all values by the sum of values.

If passed ‘all’ or True, will normalize over all values.

If passed ‘index’ will normalize over each row.

If passed ‘columns’ will normalize over each column.

If margins is True, will also normalize margin values.

Returns
DataFrame
Cross tabulation of the data.

```
나중에 실습해보기 -> 크로스탭 <https://frhyme.github.io/python-libs/pd_crosstab/>

In [ ]:
df_train[[pd.crosstab(df_train['Pclass'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

- 그룹이 된 객체에 평균을 내면 -> 각 클래스당 생존률을 얻을 수 있다. 
- 예를 들어 class1 이면 $${80 \over (80 + 136)} \approx 0.63$$ 

In [ ]:
# 평균 낸거 
df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True).mean()

In [ ]:
# 한눈에 확인하려고 변수에 담았다.
pclass_mean= df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True).mean()

pclass_mean.sort_values(by='Survived', ascending=False)
pclass_mean.sort_values(by='Survived', ascending=False).plot.bar()

- Pcalss가 좋을수록 생존률이 높은것을 확인 가능하다. 
- 시본의 카운트 폴렛을 이용하여 특정 레이블에 따른 개수를 확인해보자 

In [ ]:
# y_position = 1.02
f, ax = plt.subplots(1, 2, figsize=(18, 8)) # 틀 
pcalss = df_train['Pclass'].value_counts()
pcalss.plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass', y=y_position)
ax[0].set_ylabel('Count')
# 시본을 이용해 그림 
sns.countplot('Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()

- 클래스가 높을수록, 생존 확률이 높은것을 확인 가능하다. 
    - pclass 1,2,3 순서대로 63%, 48%, 25% 
- 이후 생존에 pcalss가 어떠한 큰영향을 준다고 생각해 볼수 있다. 
    - 모델을 빌드할때 이 feature를 사용하는것이 좋을것이라 판단 할수 있다.  

# 2.2 성별(Sex)

- 이번에는 성별로 생존률이 어떻게 달라지는지 확인 해보고자 한다. 
- 이전과 마찬가지로 pandas groupby 와 seaborn countplot 을 사용

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
Sex_mean= df_train[['Sex','Survived']].groupby(['Sex'], as_index=True).mean()
# ax[0]
Sex_mean.sort_values(by='Survived', ascending=False).plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
# ax[1]
sns.countplot('Sex', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Sex: Survived vs Sex', y=1.02)

plt.show()

- 여자의 생존률이 높은것을 확인 

In [ ]:
Sex_mean= df_train[['Sex','Survived']].groupby(['Sex'], as_index=False).mean()
Sex_mean

In [ ]:
Sex_mean.sort_values(by='Survived', ascending=False)

In [ ]:
# 크로스 탭 
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

- Pclass 와 마찬가지로, Sex 도 예측 모델에 쓰일 중요한 feature라는 것을 확인 

# 2.3 성별과 클래스(Both Sex and Pclass)

- Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는 지 확인할것임 